In [76]:
# Let's read the data_file into memory
spect_data = data.frame(read.csv("/home/srinivas/Git-Projects/spect-team/Data/Data_Adults_1.csv"))

# Drop NAs
spect_data = na.omit(spect_data)

# Extract the baseline and Concentration Values
interesting_baseline_column_start = which(colnames(spect_data) == "Baseline_Amygdala_L")
interesting_baseline_column_end = which(colnames(spect_data) == "Baseline_Vermis_9")
interesting_concentration_column_start = which(colnames(spect_data) == "Concentration_Amygdala_L")
interesting_concentration_column_end = which(colnames(spect_data) == "Concentration_Vermis_9")
baseline_values = spect_data[interesting_baseline_column_start : interesting_baseline_column_end]
concentration_values = spect_data[interesting_concentration_column_start : interesting_concentration_column_end]

# Prefix the Patient Id to make them uniquely recongizable
b_vals = cbind(spect_data["Patient_ID"], baseline_values)
c_vals = cbind(spect_data["Patient_ID"], concentration_values)

# Let us also compute the deltas in the blood flow
delta_values = concentration_values - baseline_values
deltas = cbind(spect_data["Patient_ID"], delta_values)

#deltas[1:4, 1:4]

Now that we have the the RCBF value let us read in 2 lists, a list of healthy patients and a list of people who have anxiety disorders

** EDIT THE FILE PATHS TO POINT TO A FILE ON YOUR PC **

In [77]:
healthy_patients = data.frame(read.csv("/home/srinivas/Git-Projects/spect-team/Data/healthy_IDs.csv"))
anxious_patients = data.frame(read.csv("/home/srinivas/Git-Projects/spect-team/Data/AnxietyDisorder_IDs.csv"))
print(nrow(healthy_patients))
print(nrow(anxious_patients))

We have 50 healthy people and 15 people with Anxiety Disorder, let us now get the RCBF values for these individuals

In [78]:
# Let us perform a cartesian join of the two data frames
healthy = deltas[deltas$Patient_ID %in% healthy_patients$Patient_ID,]
anxious = deltas[deltas$Patient_ID %in% anxious_patients$Patient_ID,]

Now we don't know which rcbf delta corresponds is an indicator that a person suffers from a certain disorder,
We propose running a series of Wilcoxon-Mann-Whitney rank sum test on all the columns

In [80]:
p_vector = 0
# Lets loop over the columns
for (i in names(healthy[-1])) {
    col1 = as.vector(healthy[,i])
    col2 = as.vector(anxious[,i])
    ans = wilcox.test(col1, col2, alternative="two.sided")
    p_vector = cbind(p_vector, ans[3])
}
names(p_vector) = names(healthy)
write.csv(healthy, "healthy.csv")
write.csv(anxious, "anxious.csv")
write.csv(rbind(names(healthy),p_vector), "pvals.csv")

Based, on analysisng the generated p-values it would seem that that"Cerebelum_7b_L" can be used as a differentiator to separate people with anxiety disorder from healthy people. This [Article](http://www.amenclinics.com/the-science/spect-gallery/anxiety/) says that people with anxiety have abnormal blood flow in the basal ganglia, which along with certain portions of the cerebellum, is responsible for movement and coordination